In [1]:
%pip install langchain

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/61.0 kB ? eta -:--:--
     ------ --------------------------------- 10.2/61.0 kB ? eta -:--:--
     ------------------------- ------------ 41.0/61.0 kB 653.6 kB/s eta 0:00:01
     -------------------------------------- 61.0/61.0 kB 651.3 kB/s eta 0:00:00
     ---------------------------------------- 0.0/84.4 kB ? eta -:--:--
     ---- ----------------------------------- 10.2/84.4 kB ? eta -:--:--
     ------------------ ------------------- 41.0/84.4 kB 495.5 kB/s eta 0:00:01
     ------------------------------------ - 81.9/84.4 kB 657.6 kB/s eta 0:00:01
     -------------------------------------- 84.4/84.4 kB 593.7 kB/s eta 0:00:00
     ---------------------------------------- 0.0/50.7 kB ? eta -:--:--
     -------- ------------------------------- 10.2/50.7 kB ? eta -:--:--
     -------------------------------------- 50.7/50.7 kB 518.6 kB/s eta 0:00:00
   ----

In [2]:
%pip install pinecone-client==2.2.4
%pip install pypdf
%pip install -q google-generativeai

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/57.6 kB ? eta -:--:--
     ---------------------------- ----------- 41.0/57.6 kB 1.9 MB/s eta 0:00:01
     -------------------------------------- 57.6/57.6 kB 751.5 kB/s eta 0:00:00
   ---------------------------------------- 0.0/179.4 kB ? eta -:--:--
   ------------- -------------------------- 61.4/179.4 kB 1.1 MB/s eta 0:00:01
   ------------------------------- -------- 143.4/179.4 kB 1.7 MB/s eta 0:00:01
   ---------------------------------------- 179.4/179.4 kB 1.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/307.7 kB ? eta -:--:--
   ------- -------------------------------- 61.4/307.7 kB 3.4 MB/s eta 0:00:01
   --------------- ------------------------ 122.9/307.7 kB 1.8 MB/s eta 0:00:01
   ------------------------- -------------- 194.6/307.7 kB 2.0 MB/s eta 0:00:01
   ------------------------------ --------- 235.5/307.7 kB 1.6 MB/s et

In [57]:
from langchain.llms import GooglePalm
from langchain.vectorstores import Pinecone
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.embeddings import GooglePalmEmbeddings
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

import pinecone
import os
import sys

In [58]:
import os
from dotenv import load_dotenv, dotenv_values

load_dotenv()

True

In [59]:
!mkdir pdfs

A subdirectory or file pdfs already exists.


In [60]:
loader = PyPDFDirectoryLoader("pdfs")
data = loader.load()

In [61]:
print(data)

[]


In [62]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 200
)

In [63]:
text_chunks = text_splitter.split_documents(data)

In [64]:
len(text_chunks)

0

## ***GEMINI, LLMS, PALM, PINECONE***

In [65]:
embedding = GooglePalmEmbeddings()

In [66]:
query_result = embedding.embed_query("Shivam Kumar")
print(len(query_result))

768


In [67]:
pinecone.init(
    api_key = os.getenv("PINECONE_API_KEY"),
    environment = os.getenv("PINECONE_API_ENV")
)

index_name = "langchainbot"

In [68]:
docsearch = Pinecone.from_texts([t.page_content for t in text_chunks], embedding = embedding, index_name = index_name)

In [69]:
query = "What are the Applications of Machine Learning?"

In [70]:
docs = docsearch.similarity_search(query, k=4)

In [71]:
docs

[Document(page_content='APPLICATIONS OF  MACHINE LEARNING IN INDUSTRIES  \nMODULE 1  \n1.1 APPLICATION OF MACHINES LEARNING IN BANKING \nAND SECURITIES  \n \n \n1.1.1 USAGE OF  MACHINE LEARNING IN BANKING SECTORS : \nInformation is the 21st Century gold, and financial institutions are aware of this. Armed with machine \nlearning and artificial intelligence technologies, they have the opportunity to analyze data that originates \nbeyond the bank office. Financial companies collect and store more and more user data in order to revise \ntheir strategies, improve user experience, prevent fraud, and mitigate risk s'),
 Document(page_content="1.2 APPLICATION OF MACHINE LEARNING IN \nCOMMUNICATION, MEDIA AND ENTERTAINMENT  \n \n \nMachine Learning (ML) has seen wide application in various industries, including \ncommunication, media, and entertainment. The use of ML algorithms and techniques in these \nfields has led to significant advancements in areas such as content recommendation, \nperso

### ***Now using GOOGLE PALM to StreamLine these Contents***

# **GOOGLE PALM**

In [72]:
from langchain.llms.google_palm import GooglePalm

In [73]:
llm = GooglePalm(google_api_key = os.getenv("api_key"), temperature = 0.7)

In [74]:
QA = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.as_retriever())

In [75]:
query1 = "What is Bank Fraud?"

In [76]:
QA.run(query1)

'Bank fraud is a wide range of illicit actions such as identity theft, credit card fraud, and financial or insurance fraud.'

In [77]:
while True:
  user_input = input("Input Prompt: ")
  if user_input == 'exit':
    print("Exit..")
    sys.exit()
  if user_input == '':
    continue

  result = QA({'query': user_input})
  print("Answer: ", {result['result']})

Answer:  {'Artificial neural networks can learn to represent input data in a lower-dimensional space, which can then be used to make predictions.'}
Answer:  {'The advantages of using GOOGLE PALM, LLMS, and PINCORE in a project are as follows:\n\n* **GOOGLE PALM** can help to improve the accuracy of predictions by providing a more comprehensive view of the data.\n* **LLMS** can help to identify patterns in the data that may not be obvious to human analysts.\n* **PINCORE** can help to visualize the data and identify relationships between different variables.\n\nThese tools can be used together to create a more comprehensive understanding of the data and to make more informed decisions.'}
Exit..


SystemExit: 

C:\Users\lifes\AppData\Roaming\Python\Python312\site-packages\IPython\core\interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
